In [1]:
import re
import time
import pickle
import numpy as np
import scipy
import pandas as pd
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'scipy'

In [3]:
!pip install pandas 

In [2]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
animes_df = pd.read_csv("../anime_recommendations/src/data/anime_data.csv", encoding="utf-8")

NameError: name 'pd' is not defined

In [4]:
#eliminate that anoying ending message
animes_df['synopsis'] = animes_df.synopsis.apply(lambda x: re.sub(r" \[Written by MAL Rewrite\]", "", str(x)))
#elimnitate any anime with a synopsis that has less than 2 sentences
animes_df = animes_df[animes_df.synopsis.map(lambda x: len(x.split(". ")) >= 2)]

In [5]:
synopsis_corpus = [synopsis.split(". ") for synopsis in animes_df.synopsis.tolist()]
codes_list = [code for code in animes_df.code.tolist()]

In [15]:
corpus_embeddings = []
for synopsis in synopsis_corpus:
    synopsus_embedding = []
    for sentence in synopsis:
        synopsus_embedding.append(model.encode(sentence))
    corpus_embeddings.append(sum(synopsus_embedding))    

It took: 2178.2604088783264


In [16]:
query = synopsis_corpus[6]
embeddings = []
for sentence in query:
    embeddings.append(model.encode(sentence))
embedding = sum(embeddings)

In [17]:
closest_n = 5
distances = scipy.spatial.distance.cdist([embedding], corpus_embeddings, "cosine")[0]

results = zip(range(len(distances)), distances)
results = sorted(results, key=lambda x: x[1])

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")

for idx, distance in results[1:closest_n]:
    print("https://myanimelist.net/anime/" + str(codes_list[idx]), "(Score: %.4f)" % (1-distance))





Query: When an Emishi village is attacked by a fierce demon boar, the young prince Ashitaka puts his life at stake to defend his tribe. With its dying breath, the beast curses the prince's arm, granting him demonic powers while gradually siphoning his life away. Instructed by the village elders to travel westward for a cure, Ashitaka arrives at Tatara, the Iron Town, where he finds himself embroiled in a fierce conflict: Lady Eboshi of Tatara, promoting constant deforestation, stands against Princess San and the sacred spirits of the forest, who are furious at the destruction brought by the humans. As the opposing forces of nature and mankind begin to clash in a desperate struggle for survival, Ashitaka attempts to seek harmony between the two, all the while battling the latent demon inside of him. Princess Mononoke is a tale depicting the connection of technology and nature, while showing the path to harmony that could be achieved by mutual acceptance.

Top 5 most similar sentence

In [18]:
np.save(open ("./synopsis_embeddings.npy", 'wb'), corpus_embeddings)

In [7]:
pickle.dump(codes_list, open("./anime_codes.pkl", "wb"))